#Unidad 12 Implementacion WEb

* **Part 13.1: Flask and Deep Learning Web Services** [[Video]](https://www.youtube.com/watch?v=H73m9XvKHug&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_13_01_flask.ipynb)

* Part 13.2: Using a Keras Deep Neural Network with a Web Application  [[Video]](https://www.youtube.com/watch?v=OBbw0e-UroI&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_13_03_web.ipynb)

# Flask and Deep Learning Web Services

Suppose you would like to create websites based on neural networks. In that case, the neural network must be exposed in a way that can be efficiently executed by Python and other programming languages.  The usual means for such integration is a web service. One of the most popular libraries for doing this in Python is [Flask](https://palletsprojects.com/p/flask/). This library allows you to quickly deploy your Python applications, including TensorFlow, as web services.

Neural network deployment is a complex process, usually carried out by a company's [Information Technology (IT) group](https://en.wikipedia.org/wiki/Information_technology).  When large numbers of clients must access your model, scalability becomes essential.  The cloud usually handles this.  The designers of Flask did not design for high-volume systems.  When deployed to production, you will usually wrap models in [Gunicorn](https://gunicorn.org/) or TensorFlow Serving.  We will discuss high volume cloud deployment in the next section.  Everything presented in this part ith Flask is directly compatible with the higher volume Gunicorn system. It is common to use a development system, such as Flask, when developing your initial system.

### Flask Hello World

It is uncommon to run Flask from a Jupyter notebook.  Flask is the server, and Jupyter usually fills the role of the client.  However, we can run a simple web service from Jupyter.  We will quickly move beyond this and deploy using a Python script (.py).  Because we must use .py files, it won't be easy to use Google CoLab, as you will be running from the command line.  For now, let's execute a Flask web container in Jupyter.

In [7]:
from werkzeug.wrappers import Request, Response
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/May/2021 14:09:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2021 14:09:52] "GET /favicon.ico HTTP/1.1" 404 -


This program starts a web service on port 9000 of your computer.  This cell will remain running (appearing locked up).  However, it is merely waiting for browsers to connect.  If you point your browser at the following URL, you will interact with the Flask web service.

* http://localhost:9000/

You should see Hello World displayed.

### MPG Flask

Usually, you will interact with a web service through JSON.  A program will send a JSON message to your Flask application, and your Flask application will return a JSON.  Later, in module 13.3, we will see how to attach this web service to a web application that you can interact with through a browser.  We will create a Flask wrapper for a neural network that predicts the miles per gallon.  The sample JSON will look like this.

```
{
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}
```

We will see two different means of POSTing this JSON data to our web server.  First, we will use a utility called [POSTman](https://www.getpostman.com/).  Secondly, we will use Python code to construct the JSON message and interact with Flask. 

First, it is necessary to train a neural network with the MPG dataset.  This technique is very similar to what we've done many times before.  However, we will save the neural network so that we can load it later.  We do not want to have Flask train the neural network.  We wish to have the neural network already trained and deploy the already prepared .H5 file to save the neural network.  The following code trains an MPG neural network.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics


In [9]:

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, \
        verbose=1, mode='auto',\
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),\
          callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
10/10 - 3s - loss: 1776619.8750 - val_loss: 1333536.8750
Epoch 2/1000
10/10 - 0s - loss: 1108507.7500 - val_loss: 784579.1875
Epoch 3/1000
10/10 - 0s - loss: 637181.0000 - val_loss: 438506.7188
Epoch 4/1000
10/10 - 0s - loss: 348202.4375 - val_loss: 226708.6562
Epoch 5/1000
10/10 - 0s - loss: 174476.5312 - val_loss: 108180.9766
Epoch 6/1000
10/10 - 0s - loss: 84057.8438 - val_loss: 51107.0391
Epoch 7/1000
10/10 - 0s - loss: 40227.9141 - val_loss: 23657.6094
Epoch 8/1000
10/10 - 0s - loss: 18961.0469 - val_loss: 10783.8301
Epoch 9/1000
10/10 - 0s - loss: 8777.3545 - val_loss: 5003.2520
Epoch 10/1000
10/10 - 0s - loss: 4226.0400 - val_loss: 2266.0828
Epoch 11/1000
10/10 - 0s - loss: 1982.7299 - val_loss: 991.9596
Epoch 12/1000
10/10 - 0s - loss: 936.8302 - val_loss: 430.5019
Epoch 13/1000
10/10 - 0s - loss: 447.9711 - val_loss: 216.6959
Epoch 14/1000
10/10 - 0s - loss: 247.7011 - val_loss: 147.9922
Epoch 15/1000
10/10 - 0s - loss: 170.2411 - val_loss: 133.5070
Epoch 16/1000


Next, we evaluate the score.  This evaluation is more of a sanity check to ensure the code above worked as expected. 

In [10]:
pred = model.predict(x_test)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 11.554520853905197


Next, we save the neural network to a .H5 file.

In [11]:
model.save(os.path.join("./dnn/","mpg_model.h5"))

We want the Flask web service to check that the input JSON is valid.  To do this, we need to know what values we expect and what their logical ranges are.  The following code outputs the expected fields, their ranges, and packages all of this information into a JSON object that you should copy to the Flask web application.  This code allows us to validate the incoming JSON requests.

In [12]:
cols = [x for x in df.columns if x not in ('mpg','name')]

print("{")
for i,name in enumerate(cols):
    print(f'"{name}":{{"min":{df[name].min()},\
          "max":{df[name].max()}}}{"," if i<(len(cols)-1) else ""}')
print("}")

{
"cylinders":{"min":3,          "max":8},
"displacement":{"min":68.0,          "max":455.0},
"horsepower":{"min":46.0,          "max":230.0},
"weight":{"min":1613,          "max":5140},
"acceleration":{"min":8.0,          "max":24.8},
"year":{"min":70,          "max":82},
"origin":{"min":1,          "max":3}
}


Finally, we set up the Python code to call the model for a single car and get a prediction.  You should also copy this code to the Flask web application.

In [13]:
import os
from tensorflow.keras.models import load_model
import numpy as np

model = load_model(os.path.join("./dnn/","mpg_model.h5"))
x = np.zeros( (1,7) )

x[0,0] = 8 # 'cylinders', 
x[0,1] = 400 # 'displacement', 
x[0,2] = 80 # 'horsepower', 
x[0,3] = 2000 # 'weight',
x[0,4] = 19 # 'acceleration', 
x[0,5] = 72 # 'year', 
x[0,6] = 1 # 'origin'


pred = model.predict(x)
float(pred[0])

-22.0653133392334

The completed web application can be found here:
    
* [mpg_server_1.py](./py/mpg_server_1.py)

You can run this server from the command line with the following command:

```
python mpg_server_1.py
```

If you are using a virtual environment (described in Module 1.1), make sure to use the ```activate tensorflow``` command for Windows or ```source activate tensorflow``` for Mac before executing the above command.

### Flask MPG Client

Now that we have a web service running, we would like to access it.  This server is a bit more complicated than the "Hello World" web server we first saw in this part.  The request to display was an HTTP GET.  We must now do an HTTP POST.  To accomplish access to a web service, you must use a client.  We will see how to use [PostMan](https://www.getpostman.com/) and directly through a Python program in Jupyter.

We will begin with PostMan.  If you have not already done so, install PostMan.  

To successfully use PostMan to query your web service, you must enter the following settings:

* POST Request to http://localhost:5000/api/mpg
* RAW JSON and paste in JSON from above
* Click Send and you should get a correct result

Figure 13.PM shows a successful result.

**Figure 13.PM: PostMan JSON**
![PostMan JSON](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/postman-1.png "PostMan JSON")

This same process can be done programmatically in Python.

In [21]:
import requests

json = {
  "cylinders": 8, 
  "displacement": 300,
  "horsepower": 78, 
  "weight": 3500,
  "acceleration": 20, 
  "year": 76,
  "origin": 1
}

r = requests.post("http://localhost:5000/api/mpg",json=json)
if r.status_code == 200:
    print("Success: {}".format(r.text))
else: print("Failure: {}".format(r.text))

Success: {
  "errors": [], 
  "id": "8d0274e9-36e6-4122-bff1-c1651921502f", 
  "mpg": 7.254964351654053
}



### Images and Web Services

We can also accept images from web services.  We will create a web service that accepts images and classifies them using MobileNet.  To use your neural network, you will follow the same process; load your network as we did for the MPG example. You can find the completed web service can here:

[image_server_1.py](./py/image_server_1.py)

You can run this server from the command line with:

```
python mpg_server_1.py
```

If you are using a virtual environment (described in Module 1.1), make sure to use the ```activate tensorflow``` command for Windows or ```source activate tensorflow``` for Mac before executing the above command.

To successfully use PostMan to query your web service, you must enter the following settings:

* POST Request to http://localhost:5000/api/image
* Use "Form Data" and create one entry named "image" that is a file.  Choose an image file to classify.
* Click Send and you should get a correct result

Figure 13.PMI shows a successful result.

**Figure 13.PMI: PostMan Images**
![PostMan Image](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/postman-2.png "PostMan Image")

This same process can be done programmatically in Python.

In [15]:
import requests
response = requests.post('http://localhost:5000/api/image', files=\
        dict(image=('gato.jpg',open('photos/gato.jpg','rb'))))
if response.status_code == 200:
    print("Success: {}".format(response.text))
else: print("Failure: {}".format(response.text))

Success: {
  "pred": [
    {
      "name": "tiger_cat", 
      "prob": 0.42152678966522217
    }, 
    {
      "name": "tabby", 
      "prob": 0.275607705116272
    }, 
    {
      "name": "lynx", 
      "prob": 0.23078253865242004
    }, 
    {
      "name": "Egyptian_cat", 
      "prob": 0.05514051765203476
    }, 
    {
      "name": "Persian_cat", 
      "prob": 0.0045029171742498875
    }
  ]
}

